# Reverse Logistics MVP

As a first step for my Reverse Logistics project, I will create a MVP which will show the trade lanes where this model can be applied. It will only compare the Tender itself for one customer. 

In order to know if Reverse Logistics can be applied for this customer and its lanes, it has to fulfill the following:
     * Origin City = Destination City (i.eg Lane 1 has an origin A and Lane 2 has a destination A)
     * By Definition, Origin and Destination country has to be the same
     * Annual volume has to be higher than 54, ~1 containers per week in the matching lanes.

This first MVP will exclude close cities (Google Maps API) and multiple combinations

## 1. Data exploration

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
file_root = 'Reverse_Logistics_Template.xlsx'

In [3]:
tender = pd.read_excel(file_root)

In [4]:
tender.head()

,Customer,Lane_ID,Origin_Country,Destination_Country,Origin_City,Destination_City,Origin_Adress,Destination_Adress,Equipment_Type,Unnamed: 9,Shipping_Terms,Commodity,Volume
0,CustomerA,Item-0001,FR,US,Le Havre,"New York, NY",NaN,NaN,40' Standard Dry,NaN,Quay-Quay incl. DTHC,French Wines & Spirits,3080.0
1,CustomerA,Item-0002,FR,SG,Le Havre,Singapore,NaN,NaN,40' Standard Dry,NaN,Quay-Quay incl. DTHC,French Wines & Spirits,1250.0
2,CustomerA,Item-0003,FR,US,Le Havre,"Oakland, CA",NaN,NaN,40' Standard Dry,NaN,Quay-Quay incl. DTHC,French Wines & Spirits,770.0
3,CustomerA,Item-0004,GB,KR,North West England,Busan,NaN,NaN,40' Standard Dry,NaN,Door-Quay excl. DTHC,Beer,665.0
4,CustomerA,Item-0005,FR,SG,Fos-sur-Mer,Singapore,NaN,NaN,40' Standard Dry,NaN,Quay-Quay incl. DTHC,French Wines & Spirits,610.0


In [5]:
#This will show as the Exporter Countries, United Kingdom, France and Irelande as Top3
top_exporters = tender.groupby('Origin_Country').sum()
top_exporters.sort_values('Volume', ascending=False)

,Unnamed: 9,Volume
Origin_Country,,
GB,0.0,16049.8
FR,0.0,9242.0
IE,0.0,3231.2
NL,0.0,2897.5
IT,0.0,2032.0
US,0.0,420.0
CL,0.0,308.0
TH,0.0,228.0
NZ,0.0,221.0


In [6]:
#This will show as the Importers Countries, United States, Singapore and China as Top3
top_importers = tender.groupby('Destination_Country').sum()
top_importers.sort_values('Volume', ascending=False)

,Unnamed: 9,Volume
Destination_Country,,
US,0.0,15786.1
SG,0.0,4182.4
CN,0.0,1813.4
KR,0.0,1553.0
AU,0.0,1286.8
ZA,0.0,1008.3
CO,0.0,977.0
MX,0.0,959.2
BR,0.0,843.0


## 2. Cleaning Lanes with <54 volume

In [7]:
tender_cleaned = tender[~(tender['Volume'] <54)]
tender_cleaned.head()

,Customer,Lane_ID,Origin_Country,Destination_Country,Origin_City,Destination_City,Origin_Adress,Destination_Adress,Equipment_Type,Unnamed: 9,Shipping_Terms,Commodity,Volume
0,CustomerA,Item-0001,FR,US,Le Havre,"New York, NY",NaN,NaN,40' Standard Dry,NaN,Quay-Quay incl. DTHC,French Wines & Spirits,3080.0
1,CustomerA,Item-0002,FR,SG,Le Havre,Singapore,NaN,NaN,40' Standard Dry,NaN,Quay-Quay incl. DTHC,French Wines & Spirits,1250.0
2,CustomerA,Item-0003,FR,US,Le Havre,"Oakland, CA",NaN,NaN,40' Standard Dry,NaN,Quay-Quay incl. DTHC,French Wines & Spirits,770.0
3,CustomerA,Item-0004,GB,KR,North West England,Busan,NaN,NaN,40' Standard Dry,NaN,Door-Quay excl. DTHC,Beer,665.0
4,CustomerA,Item-0005,FR,SG,Fos-sur-Mer,Singapore,NaN,NaN,40' Standard Dry,NaN,Quay-Quay incl. DTHC,French Wines & Spirits,610.0


In [11]:
rows_removed = tender['Lane_ID'].count() - tender_cleaned['Lane_ID'].count()
rows_removed

1037

In [13]:
tender_cleaned['Lane_ID'].count()
#There are 121 valid lanes for Customer A to apply Reverse Logistics to its own Tender.

121